In [13]:
import os

from langchain_community.graphs import Neo4jGraph

os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "2wsx3edc"
database = os.environ.get('NEO4J_DATABASE')
graph = Neo4jGraph(database=database)

In [14]:
from langchain_openai import AzureOpenAIEmbeddings

embedding = AzureOpenAIEmbeddings(
    model="text-embedding-ada-002",
    azure_endpoint='https://lang-chain-dev.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-05-15',
    azure_deployment='text-embedding-ada-002',
    openai_api_version='2023-05-15'
)

In [ ]:
from langchain_community.vectorstores import Neo4jVector
# ! pip3 install -U langchain-huggingface
import os
os.environ['SENTENCE_TRANSFORMERS_HOME'] = '/storage/models/embedding_models'
from langchain_huggingface import HuggingFaceEmbeddings
# Choose from https://huggingface.co/spaces/mteb/leaderboard

# embedding = HuggingFaceEmbeddings(model_name="lier007/xiaobu-embedding-v2")

model_path = os.path.join(os.environ['SENTENCE_TRANSFORMERS_HOME'], 'models--lier007--xiaobu-embedding-v2/snapshots/ee0b4ecdf5eb449e8240f2e3de2e10eeae877691')
embedding = HuggingFaceEmbeddings(model_name=model_path)

In [15]:
from langchain_community.vectorstores import Neo4jVector

lc_retrieval_query = """
WITH collect(node) as nodes
// Entity - Text Unit Mapping
WITH
collect {
    UNWIND nodes as n
    MATCH (n)<-[:HAS_ENTITY]->(c:__Chunk__)
    WITH c, count(distinct n) as freq
    RETURN c.content AS chunkText
    ORDER BY freq DESC
    LIMIT $topChunks
} AS text_mapping,
// Entity - Report Mapping
collect {
    UNWIND nodes as n
    MATCH (n)-[:IN_COMMUNITY]->(c:__Community__)
    WHERE c.summary is not null
    WITH c, c.rank as rank, c.weight AS weight
    RETURN c.summary 
    ORDER BY rank, weight DESC
    LIMIT $topCommunities
} AS report_mapping,
// Outside Relationships 
collect {
    UNWIND nodes as n
    MATCH (n)-[r]-(m) 
    WHERE NOT m IN nodes and r.description is not null
    RETURN r.description AS descriptionText
    ORDER BY r.rank, r.weight DESC 
    LIMIT $topOutsideRels
} as outsideRels,
// Inside Relationships 
collect {
    UNWIND nodes as n
    MATCH (n)-[r]-(m) 
    WHERE m IN nodes and r.description is not null
    RETURN r.description AS descriptionText
    ORDER BY r.rank, r.weight DESC 
    LIMIT $topInsideRels
} as insideRels,
// Entities description
collect {
    UNWIND nodes as n
    match (n)
    WHERE n.description is not null
    RETURN n.description AS descriptionText
} as entities
// We don't have covariates or claims here
RETURN {Chunks: text_mapping, Reports: report_mapping, 
       Relationships: outsideRels + insideRels, 
       Entities: entities} AS text, 1.0 AS score, {} AS metadata
"""

vectorstore = Neo4jVector.from_existing_graph(embedding=embedding, 
                                    index_name="description_embedding",
                                    node_label='__Entity__', 
                                    embedding_node_property='description_embedding', 
                                    text_node_properties=['description'],
                                    retrieval_query=lc_retrieval_query)
topChunks = 3
topCommunities = 3
topOutsideRels = 10
topInsideRels = 10
topEntities = 10
os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "2wsx3edc"

local_search_retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={'score_threshold': 0.9,
                   'k': topEntities,
                   'params': {
                        "topChunks": topChunks,
                        "topCommunities": topCommunities,
                        "topOutsideRels": topOutsideRels,
                        "topInsideRels": topInsideRels,
                    }},
)
local_search_retriever

VectorStoreRetriever(tags=['Neo4jVector', 'AzureOpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.neo4j_vector.Neo4jVector object at 0x173e396d0>, search_type='similarity_score_threshold', search_kwargs={'score_threshold': 0.9, 'k': 10, 'params': {'topChunks': 3, 'topCommunities': 3, 'topOutsideRels': 10, 'topInsideRels': 10}})

In [16]:
print(local_search_retriever.invoke('保險費暨保險單借款利息自動轉帳付款授權')[0].page_content)

Entities:
- 轉帳扣款作業需經核印成功且同意承保後始得進行。
- 首期保險費是保戶首次繳交的保險費用，採用金融機構轉帳作業。
- 需至各銀行臨櫃留存印鑑，核印成功後便能授權扣款。
- 因核印失敗、請款失敗或主動異動繳費管道者，其生效日認定依「新契約首期保險費繳費管道異動之繳費日認定原則」辦理。
- 不得為空白授權，應依是否取得保單號碼填寫相關欄位，未填寫完整須重新檢附授權書
- 無法授權扣款。
- 依各險種投保規則辦理
- 需填寫授權書，繳款人為非保單關係人時須檢附關係證明文件。相關規定請參照繳費方式及受理應檢附文件。
- 核印和扣款時間需參照各金融機構的相關規定。
- 須於授權書要保人欄位簽章並與要保書簽章相同
Reports:
- 在同一社區中，有兩個重要的節點和一個關鍵的關係。節點包括：

1. 一個沒有特定類型的節點，描述的是「新契約首期保險費繳費管道異動之繳費日認定原則」。
2. 一個類型為「概念」的節點，描述的是「新契約」。具體內容為：因核印失敗、請款失敗或主動異動繳費管道者，其生效日認定依「新契約首期保險費繳費管道異動之繳費日認定原則」辦理。

這兩個節點之間存在一個關係：「新契約」依照「新契約首期保險費繳費管道異動之繳費日認定原則」來進行生效日認定。
- 在同一個社區中，有四個節點和三個關係。四個節點分別是「土地銀行」、「第一銀行」、「合作金庫」和「數位帳戶」。其中「土地銀行」、「第一銀行」和「合作金庫」都是組織類型的節點，且這三個節點都有無法授權扣款的描述。「數位帳戶」則是概念類型的節點，其描述為需要至各銀行臨櫃留存印鑑，核印成功後便能授權扣款。

在關係部分，「數位帳戶」與「土地銀行」、「第一銀行」以及「合作金庫」之間均存在無法授權扣款的關係。具體來說，數位帳戶無法在「土地銀行」、「第一銀行」和「合作金庫」授權扣款。
- 在同一社區中，有兩個重要的節點和一個關鍵的關係。節點包括：

1. 一個沒有特定類型的節點，描述的是「新契約首期保險費繳費管道異動之繳費日認定原則」。
2. 一個類型為「概念」的節點，描述的是「新契約」。具體內容為：因核印失敗、請款失敗或主動異動繳費管道者，其生效日認定依「新契約首期保險費繳費管道異動之繳費日認定原則」辦理。

這兩個節點之間存在一個關係：「新契約」依照「新契約首期保險費繳費管道異動之繳費日認定原則」來

In [17]:
vectorstore = Neo4jVector.from_existing_graph(embedding=embedding, 
                                    index_name="chunk_index",
                                    node_label='__Chunk__', 
                                    embedding_node_property='embedding', 
                                    text_node_properties=['content'])
vector_retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={'score_threshold': 0.9}
)

In [18]:
import os
# from langchain_openai import ChatOpenAI
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)
llm

AzureChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x1673dbfe0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x173881310>, openai_api_key=SecretStr('**********'), openai_proxy='', azure_endpoint='https://lang-chain-dev.openai.azure.com/', deployment_name='langchain-dev', openai_api_version='2024-02-15-preview', openai_api_type='azure')

In [26]:
from langchain_ollama import ChatOllama
llm = ChatOllama(
    # model="llama3.1:70b-instruct-q8_0",
    model='qwen2:72b-instruct-q8_0',
)
llm

In [19]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template(
"""
你是一個有用的助手, 你的任務是整理提供的資訊, 使用長度與格式符合「multiple paragraphs」來回應使用者的問題",
提供的資訊包含 檢索內容、圖譜資料庫相關節點與關係資訊 
你必須使用繁體中文回應問題, 盡可能在500字內回應,
如果提供的資訊沒有答案或是皆無關聯, 請直接說「找不到相關資訊」並結束, 不要捏造任何資訊",
最終的回應將清理後的訊息合併成一個全面的答案，針對回應的長度和格式對所有關鍵點和含義進行解釋
根據回應的長度和格式適當添加段落和評論。以Markdown格式撰寫回應。
回應應保留原有的意思和使用的情態動詞，例如「應該」、「可以」或「將」。
請確保使用繁體中文回答問題

以下為檢索內容:
"{context}"

以下為圖譜資料庫相關節點(Entities)、關係(Relationships)、社群(Reports)、Chunks(內文節錄)資訊:
"{graph_result}"

問題: {input}
"""
)

rag_chain = (
    {"context": vector_retriever, "input": RunnablePassthrough(), "graph_result": local_search_retriever}
    | prompt
    | llm
    | StrOutputParser()
)

In [73]:
for r in rag_chain.stream('你對個人保險首續期繳費了解多少?'):
    print(r, end="", flush=True)

### 個人保險首續期繳費的相關規定與作業

根據所提供的資料，個人保險的首期繳費主要採用金融機構轉帳作業，並有一些相關的注意事項和規定。以下是詳細的解釋：

#### 首期保險費繳交方式
首期保險費是保戶首次繳交的保險費用，通常由保戶約定於保險公司簽約的金融機構所開立的活期帳戶進行扣款。申請金融機構轉帳繳交首期保險費時，保戶需填寫授權書，且授權書不能為空白授權。授權書應依是否取得保單號碼填寫相關欄位，未填寫完整須重新檢附授權書。

#### 授權書的規定
授權書須送予授權的金融機構核對授權人身分證統一編號及印鑑或簽章樣式。繳款人若為非保單關係人，還需檢附關係證明文件。相關規定請參照自動轉帳及信用卡付款授權書作業規範，這些規範旨在加強身份驗證機制。

#### 核印與扣款
轉帳扣款作業需經核印成功且同意承保後始得進行。核印和扣款時間需參照各金融機構的相關規定。在某些情況下，如數位帳戶，需至各銀行臨櫃留存印鑑，核印成功後便能授權扣款。不過，有些銀行如土地銀行、合作金庫、第一銀行無法授權扣款。

#### 生效日認定
保單的生效日認定依「生效日期之認定原則」辦理。若因核印失敗、請款失敗或新契約於核保中主動異動繳費管道者，其生效日認定依「新契約首期保險費繳費管道異動之繳費日認定原則」辦理。

#### 保費折扣
保費折扣的適用規定依各險種投保規則辦理。若以金融機構轉帳繳交保險費，可以享受相應的保費折扣。

以上是個人保險首續期繳費的主要內容，涵蓋了繳費方式、授權書規範、核印與扣款程序、生效日認定以及保費折扣等方面的詳細規定。這些規定旨在確保繳費過程的順利進行並保障保戶的權益。

In [20]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


contextualize_q_system_prompt = (
    "Given a chat history and the latest user question " # 給定一段聊天歷史和使用者的最新問題
    "which might reference context in the chat history, " # 這個問題可能會引用聊天歷史中的上下文
    "formulate a standalone question which can be understood " # 請將問題重新表述為一個獨立的問題，使其在沒有聊天歷史的情況下也能被理解
    "without the chat history. Do NOT answer the question, " # 不要回答這個問題
    "just reformulate it if needed and otherwise return it as is." # 只需在必要時重新表述問題，否則原樣返回
)

# 定義上下文解析的Chain
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("history"),
        ("human", "{input}"),
    ]
)

contextualize_chain = (
    contextualize_q_prompt
    | llm
    | StrOutputParser()
)


from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnableParallel, RunnableLambda

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# 使用 RunnableParallel 來組織多個並行查詢
context_and_search_chain = RunnableParallel(
    {
        "context": RunnableLambda(lambda inputs: vector_retriever.invoke(inputs)),
        "graph_result": RunnableLambda(lambda inputs: local_search_retriever.invoke(inputs)),
        "input": lambda x: x,  # 保留原始輸入
    }
)

rag_chain = (
    contextualize_chain
    | context_and_search_chain
    | prompt
    | llm
    | StrOutputParser()
)


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)


In [21]:

for r in conversational_rag_chain.stream(
    {"input": "常見的繳費方式為何有這三種?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
):
    print(r, end="",flush=True)

根據提供的資料，常見的繳費方式主要包括以下幾種：

### 金融機構轉帳
首期保險費通常採用金融機構轉帳作業。保戶可以在簽約的金融機構開立活期帳戶，並透過該帳戶進行扣款。這種方式需要保戶填寫並提交授權書，授權書的內容必須完整且不得為空白授權。授權書需送交至金融機構進行核印，核印成功後才能進行轉帳扣款。

### 授權書的使用
在使用金融機構轉帳的同時，需要填寫「保險費暨保險單借款利息自動轉帳付款授權書」。如果繳款人不是保單關係人，則需檢附關係證明文件。授權書上要保人須於授權書要保人欄位簽章，並且簽章需與要保書中的簽章相同。此外，授權人還需留存金融機構的印鑑或簽章樣式，以便金融機構核對授權人的身份證統一編號及印鑑/簽章。

### 數位帳戶繳費
數位帳戶也是一種常見的繳費方式。保戶需要到各個銀行臨櫃留存印鑑，並且核印成功後才能授權扣款。然而，需要注意的是，某些銀行如土地銀行、合作金庫、第一銀行無法進行授權扣款。

### 保費折扣
在適用某些繳費方式時，保費折扣的規定會依各險種投保規則辦理。這意味著不同的保險種類可能有不同的折扣政策，保戶需要根據所投保的險種來了解具體的折扣適用規定。

### 生效日認定
保單的生效日認定需要依照「生效日期之認定原則」辦理。如果因核印失敗、請款失敗或新契約在核保中主動異動繳費管道，則其生效日認定會依「新契約首期保險費繳費管道異動之繳費日認定原則」來辦理。

總結來說，常見的繳費方式包括金融機構轉帳、授權書繳費和數位帳戶繳費等。這些方式都需要保戶進行相關的授權和核印作業，並且在某些情況下會影響保單的生效日期及保費折扣的適用。

In [ ]:
from fastapi import FastAPI
from langserve import add_routes

# 4. App definition
app = FastAPI(
  title="LangChain Server",
  version="1.0",
  description="A simple API server using LangChain's Runnable interfaces",
)

# 5. Adding chain route

add_routes(
    app,
    conversational_rag_chain,
)

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="localhost", port=8000)